# Building a Custom ResNet for PawPularity predictions

I found it difficult to finetune a predefined ResNet(PyTorch) as we have to include the Metadata in our training as well. It looked as if we would have to overwrite the **forward(self,x)** method itself to accept two inputs i.e. **forward(self,x1,x2)**. That is exactly what I have done in this code.

Sources -
1. [Abhishek's notebook](https://www.kaggle.com/abhishek/tez-pawpular-swin-ference)
2. [Youtube](https://www.youtube.com/watch?v=qaDe0qQZ5AQ)

In [ ]:
# importing all the DL heavenly deities before we begin our code 
import torch
# For building a custom dataset and dataloader
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import cv2
import albumentations
import pandas as pd
import numpy as np
# For building a custom CNN
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class args:
    batch_size = 16
    image_size = 224


train_df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
train_img_paths = [f"../input/petfinder-pawpularity-score/train/{x}.jpg" for x in train_df["Id"].values]

class args:
    batch_size = 16
    image_size = 384
    
train_aug = albumentations.Compose(
    [
        albumentations.Resize(args.image_size, args.image_size, p=1),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

class PawPularityDataset(Dataset):
    def __init__(self,image_paths,train_df,augmentations):
        self.image_paths = image_paths
        self.train_df = train_df
        self.features = train_df.columns[1:-1]
        self.augmentations = augmentations
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self,idx):
        image = cv2.imread(self.image_paths[idx])
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        
        
        target = train_df['Pawpularity'][idx]
        
        features = list(train_df.loc[0])[1:-1]
        
        return (
            torch.tensor(image,dtype = torch.float32),
            torch.tensor(target,dtype = torch.float32),
            torch.tensor(features,dtype = torch.float32)
        )


train_loader = torch.utils.data.DataLoader(PawPularityDataset(train_img_paths,train_df,train_aug), batch_size = args.batch_size, shuffle=True)

In [ ]:

class block(nn.Module):
    def __init__(self,in_channels,out_channels,identity_downsample = None, stride = 1):
        super(block,self).__init__()

        self.expansion = 4
        self.identity_downsample  = identity_downsample

        self.convblock1 = nn.Sequential(
                nn.Conv2d(in_channels, out_channels,kernel_size = 1, stride = 1, padding =0),
                nn.BatchNorm2d(out_channels),
                nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = stride, padding =1),
                nn.BatchNorm2d(out_channels),
                nn.Conv2d(out_channels, self.expansion*out_channels, kernel_size =1,stride = 1, padding = 0),
                nn.BatchNorm2d(out_channels*self.expansion)
        )

    def forward(self,x):
        identity = x

        x = self.convblock1(x)

        if self.identity_downsample is not None :
            identity = self.identity_downsample(identity)
        
        x += identity
        x = F.relu(x)
        return x
    
class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Essentially the entire ResNet architecture are in these 4 lines below
        self.layer1 = self._make_layer(
            block, layers[0], intermediate_channels=64, stride=1
        )
        self.layer2 = self._make_layer(
            block, layers[1], intermediate_channels=128, stride=2
        )
        self.layer3 = self._make_layer(
            block, layers[2], intermediate_channels=256, stride=2
        )
        self.layer4 = self._make_layer(
            block, layers[3], intermediate_channels=512, stride=2
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(in_features = 512*4, out_features = 1024)
        self.Regressor = nn.Sequential(                              #layer where we concatenate the metadata of the image
            nn.Linear(in_features = 1024 + 12, out_features = 2000),
            nn.Linear(in_features = 2000, out_features = 1000),
            nn.Linear(in_features = 1000, out_features = 500),
            nn.Linear(in_features = 500, out_features = 100),
            nn.Linear(in_features = 100, out_features = 1)
        )

    def forward(self, x, x2):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)              # output shape = torch.Size([batch_size, 2048, 1, 1])
        x = x.reshape(x.shape[0], -1)    # output shape = torch.Size([batch_size, 2048])
        x = self.fc(x)                   # output shape = torch.Size([batch_size, 1024])
        x = x.view(-1,1024)              # output shape = torch.Size([batch_size, 1024])
        x = torch.cat((x,x2),axis = 1)   # output shape = torch.Size([batch_size, 1036])
        x = self.Regressor(x)      # output shape = torch.Size([batch_size, 1])
        x = x.view(-1,1)                 # output shape = torch.Size([batch_size, 1])

        return x

    def _make_layer(self, block, num_residual_blocks, intermediate_channels, stride):
        identity_downsample = None
        layers = []

        # Either if we half the input space for ex, 56x56 -> 28x28 (stride=2), or channels changes
        # we need to adapt the Identity (skip connection) so it will be able to be added
        # to the layer that's ahead
        if stride != 1 or self.in_channels != intermediate_channels * 4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    intermediate_channels * 4,
                    kernel_size=1,
                    stride=stride,
                    bias=False
                ),
                nn.BatchNorm2d(intermediate_channels * 4),
            )

        layers.append(
            block(self.in_channels, intermediate_channels, identity_downsample, stride)
        )

        # The expansion size is always 4 for ResNet 50,101,152
        self.in_channels = intermediate_channels * 4

        # For example for first resnet layer: 256 will be mapped to 64 as intermediate layer,
        # then finally back to 256. Hence no identity downsample is needed, since stride = 1,
        # and also same amount of channels.
        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, intermediate_channels))

        return nn.Sequential(*layers)


def ResNet50(img_channel=3, num_classes=1000):
    return ResNet(block, [3, 4, 6, 3], img_channel, num_classes)


def ResNet101(img_channel=3, num_classes=1000):
    return ResNet(block, [3, 4, 23, 3], img_channel, num_classes)


def ResNet152(img_channel=3, num_classes=1000):
    return ResNet(block, [3, 8, 36, 3], img_channel, num_classes)


def test():
    net = ResNet101(img_channel=3, num_classes=1000)
    y = net(torch.randn(4, 3, 224, 224)).to("cuda")
    print(y.size())

In [ ]:
tr = next(iter(train_loader))
tr[0]

In [ ]:
model = ResNet152()
output = model(tr[0],tr[2])
print(output)
output.shape